# Домашнее задание по курсу «Графы и сети» (МТИИ, осень 2021)

Для выполнения домашнего задания создайте дубликат этого файла прямо в Google Colab, заполните в нужных местах своим кодом, расшарьте по ссылке доступ для аккаунта dainiak@gmail.com, а саму ссылку на файл пришлите в соответствующей форме.

In [ ]:
import networkx as nx

### Задание 1
Построив соответствующую сеть и решив для неё задачу о максимальном потоке любым из [имеющихся в networkx алгоритмов](https://networkx.org/documentation/stable/reference/algorithms/flow.html), найдите паросочетание максимального размера в заданном неориентированном двудольном графе $G$ ниже. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:
k = 5
n = 20
G = nx.Graph()
G.add_nodes_from(range(2*n))
G.add_edges_from(
    (a, b + n) 
    for a,b in nx.random_regular_graph(k, n, 2020).edges()
)
# Построенный выше тестовый граф G — двудольный
# с номерами вершин в одной доле от 0 до n-1
# и в другой доле от n до 2*n-1

In [ ]:
# В переменную M сохраните множество рёбер 
# наибольшего паросочетания в графе G
# Ребро — это tuple или set или frozenset,
# содержащее соответствующую пару вершин
# В переменную M сохраните множество рёбер 
# наибольшего паросочетания в графе G
# Ребро — это tuple или set или frozenset,
# содержащее соответствующую пару вершин

D = G.copy()
left = set()
right = set()
G.add_node(2*n)
G.add_node(2*n+1)
G = nx.DiGraph()
for i in D.nodes:
  if (i <= n ):
    left.add(i)
  else:
    right.add(i)
new_nodes = set()
new_nodes.update(left)
new_nodes.update(right)
G.add_edges_from([(2*n,a) for a in range(n)])
G.add_edges_from([(b,2*n+1) for b in range(n,2*n)])
G.add_edges_from([(i, j) for i, j in zip(left, right)])

for i, j in zip(left, right):
  nx.set_edge_attributes(D, {(i, j): {"capacity": 1.0}})
for nd in G.edges():
   G[nd[0]][nd[1]]['capacity'] = 1.0

flow_value, flow_dict = nx.maximum_flow(G, 2*n, 2*n+1, capacity='capacity')
sorted_flow = sorted(flow_dict.items(), key = lambda x: len(x[1]))

M = set()
used = set()
matched = set()
for node in [x[0] for x in sorted_flow]:
  if node < n:
    for el in flow_dict[node]:
      if el not in used and el != n and el != 2*n+1:
        used.add(el)
        M.add((node,el))
        matched.add(node)
        break
# Ваш код тут.

In [ ]:
2*n in G
2*n+1 in G

True

In [ ]:
''' D = G.copy()
left = set()
right = set()
M = set()
G.add_node(-1)
G.add_node(-2)
D = nx.DiGraph()
for i in D.nodes:
  if (i <= n-1 and i >=0 or i == -1 ):
    left.add(i)
  else:
    right.add(i)
new_nodes = set()
new_nodes.update(left)
new_nodes.update(right)
D.add_nodes_from(new_nodes)
#D.add_edges_from([(i, j) for i, j in zip(left, right)])
for i, j in zip(left, right):
  nx.set_edge_attributes(D, {(i, j): {"capacity": 1.0}})
G.add_nodes_from([2*n, 2*n+1])
G.add_edges_from([(2*n,a) for a in range(n)])
G.add_edges_from([(b,2*n+1) for b in range(n,2*n)])
for e in G.edges():
    G[e[0]][e[1]]['weight'] = 1
flow_value, flow_dict = nx.maximum_flow(G, 2*n, 2*n+1, capacity='weight') '''
 '''

In [ ]:
# Это простая проверка того, что M образует паросочетание,
# и что его размер совпадает с эталонным
from functools import reduce
print(1 == len({
    2*len(M),
    len(reduce(lambda e1, e2: set(e1) | set(e2), M)),
    len(nx.bipartite.maximum_matching(G, range(n)))
}))

True


### Задание 2
Построив соответствующую сеть и решив для неё задачу о максимальном потоке, найдите максимально возможное количество не пересекающихся по внутренним **вершинам** путей между вершинами $s$ и $t$ в заданном неориентированном графе $G$. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:

def num_disjoint_paths(G, s, t):
    for nd in G.edges():
      G[nd[0]][nd[1]]['capacity'] = 1
    flow_value, flow_dict = nx.maximum_flow(G, s, t, capacity='capacity')
    return flow_value
    

    # Ваш код тут.

In [ ]:
# Простой тест, он должен проходиться при запуске непосредственно после предыдущей ячейки
G = nx.mycielski_graph(10)
s = 7
t = 700
print(
    num_disjoint_paths(G, s, t) 
    == 
    nx.node_connectivity(G, s, t)
)

True


### Задание 3
Построив поток в соответствующей сети, выберите из данного множества задач `tasks` такое подмножество, суммарная выгода выполнения задач которого максимальна. 

Словарь `prerequisites` содержит для каждой задачи перечень других задач, которые требуется решить перед началом её выполнения (если таковые есть). 

Словарь `profit` содержит информацию о выгоде решения каждой из задач. Выгоды — целочисленные, могут быть как положительные, так и отрицательные, и нулевые.

Также задано множество `compulsory` задач, которые нужно решить в обязательном порядке. Подумайте, и опишите в комментириях, как с минимальными усилиями интегрировать это в рассказанный на лекции алгоритм. Убедитесь, что Ваш код проходит тест ниже.

In [ ]:
# Ниже приведён алгоритм решения задачи полным перебором,
# результат работы которого можно считать эталонным
from itertools import combinations

def brute_force_selection(tasks, prerequisites, profit, compulsory):
    best_profit = 0
    best_selection = set()
    for ss in range(1, len(tasks)):
        for c in combinations(tasks, ss):
            if any(t not in c for t in compulsory):
                continue
            for t in c:
                if t in prerequisites:
                    for p in prerequisites[t]:
                        if p not in c:
                            break
                    else:
                        continue
                    break
            else:
                current_profit = sum(profit[t] for t in c)
                if current_profit > best_profit:
                    best_profit = current_profit
                    best_selection = set(c)
    return best_selection

In [ ]:
import numpy as np

def flow_based_selection(tasks, prerequisites, profit, compulsory):
  G = nx.DiGraph()
  #best_profit = 0
  #best_selection = set()
  G.add_nodes_from(tasks)
  G.add_nodes_from(['s','t'])
  G.add_edges_from([(u,v) for u in prerequisites for v in prerequisites[u]])
  
  for u in compulsory:
     G.add_edge('s',u)

  for u in profit:
    if profit[u] <=0:
      G.add_edge(u,'t',capacity=-profit[u])
#     elif u in compulsory:
#        continue 
    else:
        G.add_edge('s',u,capacity=profit[u])
# обязательные задачки с большой капасити
  for u in compulsory:
    G.add_edge('s',u,capacity=np.inf)

  cut_v,par = nx.minimum_cut(G,'s','t')
  return par[0] - {'s'}

    # Ваш код тут.

In [ ]:
flow_based_selection(tasks, prerequisites, profit, compulsory)

{2, 4, 5, 6, 7, 8, 9, 13, 15, 16, 17, 18, 19, 20, 21}

In [ ]:
# Простой тест, он должен проходиться при запуске непосредственно после выполнения предыдущей ячейки
import random
random.seed(2020)

tasks = set(range(1, 22))

ranges = [
    (1, len(tasks) // 3), 
    (len(tasks) // 3, 2 * len(tasks) // 3), 
    (2 * len(tasks) // 3, len(tasks))
]

prerequisites = {}
for ri, r in enumerate(ranges):
    if ri == 0:
        continue
    for t in range(*r):
        if random.randrange(1000)/1000 < 0.4:
            prerequisites[t] = set(random.sample(range(*ranges[ri-1]), 4))

profit = {
    task: random.randrange(-7, 15)
    for task in tasks
}

compulsory = set(random.sample(tasks, 2))

print(1 == len(set(
    sum(map(profit.get, 
        method(
            tasks, 
            prerequisites, 
            profit, 
            compulsory
        )
    ))
    for method in [
        brute_force_selection, 
        flow_based_selection
    ]
)))

True
